In [1]:
from datasets import load_dataset

train = '../../corpus/2-raw/train.txt'
val = '../../corpus/2-raw/val.txt'

dataset = load_dataset("text", data_files={"train": train, "validation": val})

Found cached dataset text (C:/Users/Sleepyard/.cache/huggingface/datasets/text/default-86d90c9202e113c5/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-chinese"

class TokenizerWrapper:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    
    def tokenize(self, examples):
        return self.tokenizer(examples["text"])

wrap = TokenizerWrapper(model_checkpoint)

In [4]:
tokenized_data = dataset.map(wrap.tokenize, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/20777833 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2308649 [00:00<?, ? examples/s]

In [9]:
block_size = 128

class GroupWrapper:
    def __init__(self, block_size):
        self.block_size = block_size

    def group_texts(self, examples):
        # Concatenate all texts
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder
        total_length = (total_length // self.block_size) * self.block_size
        # Split by chunks of max_len
        result = {
            k: [t[i : i + self.block_size] for i in range(0, total_length, self.block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

group = GroupWrapper(block_size)

In [10]:
lm_dataset = tokenized_data.map(
    group.group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/20777833 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2308649 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    "bert-rmrb-chinese",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    data_collator=data_collator,
)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.37


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.109144,1.919501
2,1.986566,1.884295
3,1.952879,1.861102


TrainOutput(global_step=7218, training_loss=2.0377309222603213)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.37
